<a href="https://colab.research.google.com/github/Fevia96/instagram-scraper/blob/main/instagram_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from instagrapi import Client
from instagrapi.types import StoryMention, StoryMedia, StoryLink, StoryHashtag

cl = Client()
cl.load_settings('dump_settings.json')
cl.login_by_sessionid("60187013644%3AVTtyrqRNFcVW1Z%3A9%3AAYeMryZWnzIxgJTq7hbWnYDNpLyTGHHBJw-iJy872g")


#cl.dump_settings('dump_settings.json')

True

In [37]:
#@title User

import dash
from dash import html, dcc, dash_table
from dash.dash_table.Format import Group
import base64
import urllib.request
from PIL import Image
from IPython.display import display

ACCOUNT_USERNAME = "smiech_smiech" # @param {type:"string"}

user_id = cl.user_id_from_username(ACCOUNT_USERNAME)
user_info = cl.user_info(user_id)

# Przygotowanie danych do tabeli
user_info_dict = user_info.dict()
user_info_table_data = []

# Dodaj pozostałe atrybuty do tabeli
for key, value in user_info_dict.items():
    user_info_table_data.append({"Atrybut": key, "Wartość": value})

# Przygotowanie obrazków
images = []
image_num = 0

for key, value in user_info_dict.items():
    if key == 'profile_pic_url' or key == 'profile_pic_url_hd':
        # Pobranie obrazka
        image_url = value
        image_name = f"assets/image_{image_num}.jpg"  # Zmieniamy ścieżkę
        urllib.request.urlretrieve(image_url, image_name)

        # Użycie Pillow do otwarcia obrazka
        pil_img = Image.open(image_name)

        # Konwersja obrazka do base64
        with open(image_name, "rb") as image_file:
            img_data = base64.b64encode(image_file.read())
            img_data = img_data.decode()
            img_data = f"data:image/jpg;base64, {img_data}"

        # Tworzenie komponentu obrazka
        img = html.Img(src=img_data, alt="my image", width="400")
        images.append(img)

        # Zwiększenie numeru obrazka
        image_num += 1

# Utwórz aplikację Dash
app = dash.Dash(__name__)

# Utwórz DataTable z zaznaczalnymi komórkami i eksportem do XLSX
table = dash_table.DataTable(
    id='table',
    columns=[{'id': col, 'name': col, 'editable': True} for col in user_info_table_data[0].keys()],
    data=user_info_table_data,
    css=[{'selector': 'table', 'rule': 'table-layout: fixed'}],
    style_header={
        'border': '1px solid black',
        'backgroundColor': 'rgb(22, 26, 29)',
        'color': 'white',
        'font-weight': 'bold',
        'text-align': 'center',
    },
    style_cell={
        'maxWidth': '100px',  # Ustaw maksymalną szerokość komórki na 100px
        'textOverflow': 'ellipsis',
        'overflow': 'hidden',
        'textAlign': 'left',
        'backgroundColor': 'rgb(22, 26, 29)',
        'color': 'white'
    },
    export_format='xlsx',
    export_headers='display',
)

# Dodaj przycisk eksportu
export_button = dcc.Download(id="export-data-xlsx")

# Układ strony
app.layout = html.Div(
    style={'backgroundColor': '#111111', 'color': 'white', 'maxWidth': '1200px', 'margin': 'auto'},  # Ograniczenie szerokości i centrowanie
    children=[
        html.H1('Informacje o użytkowniku', style={'color': 'white'}),
        table,
        *images,
        export_button,
    ]
)

# Uruchom aplikację Dash
if __name__ == '__main__':
    app.run_server(debug=True)



<IPython.core.display.Javascript object>